In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
#from cbomcode.tools import fits_cat as fc
#import deepPlotRegression_bernardo as plotter
import seaborn as sns
import os
import pickle
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from astropy.io import fits
import tensorflow as tf
# from knockknock import email_sender
#@email_sender(recipient_emails=["lucianadiasaqua@hotmail.com"])

In [2]:
from google.colab import drive
drive.mount('/content/drive')
dir_data_base = "drive/MyDrive" # montei no meu google colab o meu diretório do meu google drive
!ls drive/MyDrive
#https://drive.google.com/file/d/1dlJw8yIHeUlzmY66gvon0tx9nQVVoTpi/view?usp=share_link
#https://drive.google.com/file/d/1vFvGFcoxwFOxcwZ2mGJ2oX6T2m2QB_zo/view?usp=share_link
# https://drive.google.com/file/d/14WWbv6KnLIUNe9Egr_UvZPSWxWZDuE4s/view?usp=share_link


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 catalogExemplefits.csv		       iDR3.STRIPE82-0010.027444_R.fits
 CATALOGOSWGO_CLECIO_22_05_23_V2.csv   images_normalize_1000x100x100x3.npy
 classes_1000.npy		       Joao
 Classroom			       logLossEpochEMaior.csv
'Colab Notebooks'		       logLossEpochEMenor.csv
 DataColab			       logLossEpochXYCORE.csv
 DatasetIDR3.npy		       new_classificationsSplus_ID3.csv
 foo.txt			       R
 G				       Regression_random_forest_copiateste.ipynb
 I				       x_data20000fits.h5
 id2keep_1000.npy		       Ximenes
 iDR3.STRIPE82-0010.027444_G.fits      y_data20000fits.csv
 iDR3.STRIPE82-0010.027444_I.fits


In [3]:
if not os.path.exists(os.path.join(dir_data_base,"iDR3.STRIPE82-0010.027444_G.fits")):
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1dlJw8yIHeUlzmY66gvon0tx9nQVVoTpi' -O iDR3.STRIPE82-0010.027444_G.fits
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vFvGFcoxwFOxcwZ2mGJ2oX6T2m2QB_zo' -O iDR3.STRIPE82-0010.027444_I.fits
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=14WWbv6KnLIUNe9Egr_UvZPSWxWZDuE4s' -O iDR3.STRIPE82-0010.027444_R.fits

In [4]:
# construi um dicionario para passar para dataframe construindo um catálogo
# repeti o id 3 vezes para demonstrativo, mas na realidade deveriam ser 3 IDs diferentes
# onde cada um correspondem a um sample diferente
df = pd.DataFrame({'ID': np.array(['iDR3.STRIPE82-0010.027444', 'iDR3.STRIPE82-0010.027444', 'iDR3.STRIPE82-0010.027444']),
                    'classe': np.array([1,0,1])})
df.to_csv("drive/MyDrive/catalogExemplefits.csv")
df = pd.read_csv("drive/MyDrive/catalogExemplefits.csv")

In [5]:

Channels = ['G','I','R']

ID_SPLUS = df["ID"]
CLS = df['classe']
SIZE_IMG = 200
Data_NPY_IMG = os.path.join(dir_data_base, "DatasetIDR3.npy")

if not os.path.exists(Data_NPY_IMG):
  IMAGES = np.zeros((len(ID_SPLUS),SIZE_IMG,SIZE_IMG,len(Channels)))
  for m,i in enumerate(Channels):
    for n,j in enumerate(ID_SPLUS):
      filen = j + '_' + i + '.fits'
      hst = fits.open(os.path.join(dir_data_base, filen))
      header = hst[0].header
      img = hst[0].data
      #print(img.shape)
      IMAGES[n,:,:,m] = img

  np.save(os.path.join(dir_data_base, "DatasetIDR3.npy"), IMAGES)
else:
  IMAGES = np.load(Data_NPY_IMG)




In [8]:
# criando modelo
strategy = tf.distribute.MirroredStrategy()
#del inp, efn

import tensorflow
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import utils

with strategy.scope():
      inp = tf.keras.layers.Input(shape=(SIZE_IMG,SIZE_IMG,len(Channels)), name="inpSplus") # vc tá criando conforme seu dado
      efn = EfficientNetB0(input_tensor=inp, weights="imagenet") # tá carregando pesos da imagenet, para nao carregar substitua "imagenet" por None
      outputs = tf.keras.layers.Dense(CLS.max()+1, activation="softmax", name="clsSplus")(efn.layers[-2].output) # vc tá criando essa última camada conforme o q vc precisa
      model = tf.keras.Model(inp, outputs)
      model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 inpSplus (InputLayer)       [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 200, 200, 3)          0         ['inpSplus[0][0]']            
                                                                                                  
 normalization_2 (Normaliza  (None, 200, 200, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling)     (None, 200, 200, 3)          0         ['normalization_2[0][0]'

In [9]:
epochs = 10

# criando dado random pra testar o modelo
xtrain = np.random.rand(1000,SIZE_IMG, SIZE_IMG, len(Channels))
xval = np.random.rand(100,SIZE_IMG, SIZE_IMG, len(Channels))
ytrain = np.random.randint(2,size=(1000, 1))
ytrain = utils.to_categorical(ytrain, ytrain.max()+1)
yval = np.random.randint(2,size=(100, 1))
yval = utils.to_categorical(yval, yval.max()+1)
hist = model.fit(xtrain,ytrain, epochs=epochs, validation_data=(xval,yval), verbose=1)

Epoch 1/10
32/32 [==============================] - 286s 8s/step - loss: 0.7694 - accuracy: 0.4940 - val_loss: 0.7255 - val_accuracy: 0.4400
Epoch 2/10
32/32 [==============================] - 234s 7s/step - loss: 0.6201 - accuracy: 0.6630 - val_loss: 0.8038 - val_accuracy: 0.4600
Epoch 3/10
32/32 [==============================] - 226s 7s/step - loss: 0.4151 - accuracy: 0.8170 - val_loss: 25.2949 - val_accuracy: 0.5100
Epoch 4/10
32/32 [==============================] - 224s 7s/step - loss: 0.2923 - accuracy: 0.8810 - val_loss: 54.3938 - val_accuracy: 0.5500
Epoch 5/10
32/32 [==============================] - 224s 7s/step - loss: 0.2307 - accuracy: 0.8990 - val_loss: 308.9754 - val_accuracy: 0.4400
Epoch 6/10
32/32 [==============================] - 225s 7s/step - loss: 0.1433 - accuracy: 0.9410 - val_loss: 1486.5942 - val_accuracy: 0.5700
Epoch 7/10
32/32 [==============================] - 219s 7s/step - loss: 0.1775 - accuracy: 0.9290 - val_loss: 1055.1272 - val_accuracy: 0.5700
Epo